In [1]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

# Code example: popcount

Given a 64-bit integer number, find the number of 1s in its binary representation.

Example 1:

Input: 59487

Output: 9

Explanation: 9487’s binary representation is 0b10110010100001111

In [32]:
render_code("popcounts/popcount.cpp", show=["#else //A", "#endif"])

// popcounts/popcount.cpp:87-97 (11 lines)
#else //A
inline int popcount(uint64_t x){
    int c=0;
    while(x) //while any bits are set not sure how you are getting the value
    {
       c += x & 1;
       x = x >> 1;
    }
    return c;
}
#endif

In [33]:
render_code("popcounts/popcount.cpp", show=["#ifdef B", "#else"])

// popcounts/popcount.cpp:32-48 (17 lines)
#ifdef B
inline int popcount(uint64_t x) {
    int c = 0;
    while(x) //while any bits are set not sure how you are getting the value
    {
        c += x & 1;
        x = x >> 1;
        c += x & 1;
        x = x >> 1;
        c += x & 1;
        x = x >> 1;
        c += x & 1;
        x = x >> 1;
    }
    return c;
}
#else

In [34]:
render_code("popcounts/popcount.cpp", show=["#ifdef C", "#else"])

// popcounts/popcount.cpp:20-31 (12 lines)
#ifdef C
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     while(x) {
            c += table[(x & 0xF)];
            x = x >> 4;
     }
     return c;
}

#else

In [35]:
render_code("popcounts/popcount.cpp", show=["#ifdef D", "#else"])

// popcounts/popcount.cpp:49-59 (11 lines)
#ifdef D
inline int popcount(uint64_t x) {
     int c = 0;
     int table[16] = {0, 1, 1, 2, 1, 2, 2, 3, 1, 2, 2, 3, 2, 3, 3, 4};
     for (uint64_t i = 0; i < 16; i++) {
            c += table[(x & 0xF)];
            x = x >> 4;
     }
     return c;
}
#else

In [36]:
render_code("popcounts/popcount.cpp", show=["#ifdef E", "#else"])

// popcounts/popcount.cpp:60-87 (28 lines)
#ifdef E
inline int popcount(uint64_t x) {
     int c = 0;
     for (uint64_t i = 0; i < 16; i++) {
         switch((x & 0xF))
         {
             case 1: c+=1; break;
             case 2: c+=1; break;
             case 3: c+=2; break;
             case 4: c+=1; break;
             case 5: c+=2; break;
             case 6: c+=2; break;
             case 7: c+=3; break;
             case 8: c+=1; break;
             case 9: c+=2; break;
             case 10: c+=2; break;
             case 11: c+=3; break;
             case 12: c+=2; break;
             case 13: c+=3; break;
             case 14: c+=3; break;
             case 15: c+=4; break;
             default: break;
         }
         x = x >> 4;
     }
     return c;
}
#else //A

## Who has the best performance?

In [7]:
render_code("popcounts/popcount.cpp", show="main")

// popcounts/popcount.cpp:114-136 (23 lines)
int main(int argc, char *argv[]) {

     uint64_t key = 0xdeadbeef;
     char preamble[1024];
     char epilogue[1024];
     char header[1024];
     char stat_file[] = "stats.csv";
     int count = 1000000000;
     uint64_t sum = 0;
     perfstats_init();
     perfstats_enable(1);     
     for (int i=0; i < count; i++)
     { 
	   sum += popcount (RandLFSR(key)); 
     }
     perfstats_disable(1);
     sprintf(epilogue,"\n");
     sprintf(preamble,"");
     perfstats_print(preamble, stat_file, epilogue);
     perfstats_deinit();
     printf("Result: %lu\n", sum);
     return sum;
}

In [ ]:
! make -C popcounts clean; make -C popcounts
! echo "Version,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branc_miss_predict_rate,branch_misses,branches" > stats.csv
! echo "Version A"; echo -n "A," >> stats.csv; time ./popcounts/popcount_A
! echo "Version B"; echo -n "B," >> stats.csv; time ./popcounts/popcount_B
! echo "Version C"; echo -n "C," >> stats.csv; time ./popcounts/popcount_C
! echo "Version D"; echo -n "D," >> stats.csv; time ./popcounts/popcount_D
! echo "Version E"; echo -n "E," >> stats.csv; time ./popcounts/popcount_E

make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
rm -f popcount_A popcount_B popcount_C popcount_D popcount_SSE42 popcount_E
make: Leaving directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
make: Entering directory '/nfshome/htseng/courses/CS203/demo/OoO_programming/popcounts'
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -w popcount.cpp perfstats.o -o popcount_A
g++ -S -O3 -DHAVE_LINUX_PERF_EVENT_H -w popcount.cpp -o A.s
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -w -DB popcount.cpp perfstats.o -o popcount_B
g++ -S -O3 -DHAVE_LINUX_PERF_EVENT_H -w -DB popcount.cpp -o B.s
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -w -DC popcount.cpp perfstats.o -o popcount_C
g++ -S -O3 -DHAVE_LINUX_PERF_EVENT_H -w -DC popcount.cpp -o C.s
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -w -DD popcount.cpp perfstats.o -o popcount_D
g++ -S -O3 -DHAVE_LINUX_PERF_EVENT_H -w -DD popcount.cpp -o D.s
g++ -O3 -DHAVE_LINUX_PERF_EVENT_H -w -m64 -msse4.2 -DSSE42 popcount.cpp perfstats.o -o popcou

In [23]:
df = render_csv("stats.csv")
df = df.loc[df['Version'].isin(["A","B"])]
display_df_mono(df = df)

,index,Version,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,A,332805374364,117560792106,0.353242,0.194260,22.837399,0.013836,4888835,353330679,65130265488,841327546
1,1,B,288702121794,65992802815,0.228584,0.194220,12.817095,0.011856,4204973,354662200,18073460375,63379463


In [28]:
compare([do_render_code("./popcounts/A.s",show=["L6","jne	.L6"]),do_render_code("./popcounts/B.s",show=["L6","jne	.L6"])])

In [24]:
df = render_csv("stats.csv")
df = df.loc[df['Version'].isin(["A","B","C"])]
display_df_mono(df = df)

,index,Version,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,A,332805374364,117560792106,0.353242,0.194260,22.837399,0.013836,4888835,353330679,65130265488,841327546
1,1,B,288702121794,65992802815,0.228584,0.194220,12.817095,0.011856,4204973,354662200,18073460375,63379463
2,2,C,101701961417,26032426956,0.255968,0.194529,5.064057,0.000027,536801,19950074604,17939339374,67167654


In [13]:
compare([do_render_code("./popcounts/B.s",show=["L6","jne	.L6"]),do_render_code("./popcounts/C.s",show=["L6","jne	.L6"])])

In [25]:
df = render_csv("stats.csv")
df = df.loc[df['Version'].isin(["A","B","C","D"])]
display_df_mono(df = df)

,index,Version,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,A,332805374364,117560792106,0.353242,0.194260,22.837399,0.013836,4888835,353330679,65130265488,841327546
1,1,B,288702121794,65992802815,0.228584,0.194220,12.817095,0.011856,4204973,354662200,18073460375,63379463
2,2,C,101701961417,26032426956,0.255968,0.194529,5.064057,0.000027,536801,19950074604,17939339374,67167654
3,3,D,80030832426,19344490600,0.241713,0.194008,3.752994,0.000020,392116,20012744657,1006991616,21856


In [15]:
compare([do_render_code("./popcounts/C.s",show=["L6","jne	.L6"]),do_render_code("./popcounts/D.s",show=["L5","jne	.L5"])])

In [38]:
df = render_csv("stats.csv")
display_df_mono(df = df)

,index,Version,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branc_miss_predict_rate,branch_misses,branches
0,0,A,334270949858,118508036769,0.354527,0.196077,23.236715,0.013631,12673939,929754678,0.011765,769062344,65366730559
1,1,B,290179950840,68341242029,0.235513,0.196350,13.418770,0.011645,11645581,1000057197,0.003549,65012728,18319495534
2,2,C,102882218758,27580097715,0.268074,0.195071,5.380071,0.000265,5412643,20430859423,0.003731,67647126,18129328282
3,3,D,80043714833,19072124536,0.238271,0.196832,3.754005,0.000076,1514962,20048842732,0.000232,240743,1037120144
4,4,E,253238690876,376641071475,1.487297,0.196412,73.976956,0.004846,37136629,7663443283,0.183653,13584378400,73967642413


In [17]:
compare([do_render_code("./popcounts/D.s",show=["L5","jne	.L5"]),do_render_code("./popcounts/E.s",show=["L12",".LFE1593:"])])

### Hardware acceleration!

In [39]:
! echo -n "SSE," >> stats.csv; ./popcounts/popcount_SSE42
display_df_mono(render_csv("stats.csv"))

Result: 32000010811


,index,Version,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branc_miss_predict_rate,branch_misses,branches
0,0,A,334270949858,118508036769,0.354527,0.196077,23.236715,0.013631,12673939,929754678,0.011765,769062344,65366730559
1,1,B,290179950840,68341242029,0.235513,0.196350,13.418770,0.011645,11645581,1000057197,0.003549,65012728,18319495534
2,2,C,102882218758,27580097715,0.268074,0.195071,5.380071,0.000265,5412643,20430859423,0.003731,67647126,18129328282
3,3,D,80043714833,19072124536,0.238271,0.196832,3.754005,0.000076,1514962,20048842732,0.000232,240743,1037120144
4,4,E,253238690876,376641071475,1.487297,0.196412,73.976956,0.004846,37136629,7663443283,0.183653,13584378400,73967642413
5,5,SSE,22089754243,8444815558,0.382296,0.195891,1.654263,0.016113,583069,36186208,0.000081,82140,1014543318
